In [1]:
# 第一步，定义残差块
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l
device = d2l.try_gpu()

In [2]:
class Residual(nn.Module):
    def __init__(self,input_channels,num_channels,use_1x1conv=True, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,kernel_size=3, padding=1)
        # 是否对输出使用1X1卷积层
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True) 
    
    def forward(self,X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        # 这里是核心,输出残差
        # 这里需要注意的是，设计Resnet，Y的shape需要和X保持一致
        Y += X
        return F.relu(Y)

# ResNet模型
# ResNet 的前两层跟之前介绍的GoogLeNet 中的⼀样：在输出通道数为64、步幅为2 的7X7 卷积层后，接
# 步幅为2 的3X3 的最⼤汇聚层。不同之处在于ResNet 每个卷积层后增加了批量归⼀化层。
b1 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

# 这种写法之前比较少见，通过数组的形式添加层，在直接*args添加进Sequential
def resnet_block(input_channels, num_channels, num_residuals,first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))
net = nn.Sequential(
    b1, 
    b2, 
    b3, 
    b4, 
    b5, 
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten(), nn.Linear(512, 256))



In [3]:
net.load_state_dict(torch.load('resnet.params'))
net.eval()
net.to(device)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 64, k

In [4]:
import pickle
with open('./labels.pkl','rb') as file:
    labels = pickle.load(file)

In [5]:
import pandas as pd
import pickle
root = './data/predict.csv'
root = pd.read_csv(root)
image = root['image'].tolist()

In [6]:
# 数据处理
import os
import torch
from torch.utils.data import Dataset
# Augmentation
import albumentations
from albumentations.pytorch.transforms import ToTensorV2
import cv2

transform_test = albumentations.Compose([
            albumentations.Resize(320, 320),
            albumentations.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225],
                max_pixel_value=255.0, always_apply=True
            ),
            ToTensorV2(p=1.0)
        ]
    )

class Mydataset(Dataset):
    def __init__(self,image, transform):
        """
        定义自己的数据集合
        将图片转换为Tensor,归一化至[0,1]
        transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])  # 标准化至[-1,1]
        imgs 因为所有图片的绝对路径，这里路径都储存在了train.csv中
        所以这里路径应当根据csv来提供
        """
        self.imgs = [os.path.join('./data',x) for x in image]
        self.transform = transform

    def __getitem__(self, index):
        image_filepath = self.imgs[index]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        image_filepath = image_filepath.split('/')[-1]
        return image,image_filepath

    def __len__(self):
        return len(self.imgs)

In [7]:
from torch.utils.data import DataLoader
test_iter = Mydataset(image,transform=transform_test)
test_iter = DataLoader(test_iter,batch_size=64,shuffle=False,num_workers=0)

In [8]:
result = []
s,i = torch.tensor(0.0,device=device),0
for a,ind in test_iter:
    a =a.to(device)
    i+=1
    _, predicted = torch.max(net(a), 1)
    result.extend(list(zip(ind,predicted.tolist())))

In [9]:
import pandas as pd
predict_result = pd.DataFrame(result,columns=['image','label'])

In [10]:
predict_result['image'] = predict_result['image'].map(lambda x: f'images/{x}')
predict_result['label'] = predict_result['label'].map(lambda x: labels[x])

In [11]:
predict_result.to_csv('./predict_result.csv',index=False)